In [103]:
import torch
from torch import nn, Tensor

_PLUS_EPS = 1e-9

class MGIoU2DPlus(nn.Module):
    """MGIoU for arbitrary convex quadrangles with an optional convexity loss."""

    def __init__(
        self,
        convex_weight: float = 0.0,
        fast_mode: bool = False,
        reduction: str = "mean",
    ):
        super().__init__()
        if reduction not in {"none", "mean", "sum"}:
            raise ValueError("reduction must be 'none','mean' or 'sum'.")
        self.convex_weight = convex_weight
        self.fast_mode = fast_mode
        self.reduction = reduction

    def forward(
        self,
        pred: Tensor,                  # [B,4,2]
        target: Tensor,                # [B,4,2]
        visible_mask: Tensor | None = None,
        reduction_override: str | None = None,
        return_dict: bool = False,
    ) -> Tensor | tuple[Tensor, dict]:
        if visible_mask is not None:
            mask = visible_mask.bool().squeeze()
            pred, target = pred[mask], target[mask]

        # per-sample MGIoU
        mgiou_samples = torch.vmap(self._mgiou_single)(pred, target)
        losses = (1.0 - mgiou_samples) * 0.5

        diag = {"mgiou": mgiou_samples.detach()}

        # convexity penalty
        if self.convex_weight != 0.0:
            convex_term = self._convexity_loss(pred)
            losses = losses + self.convex_weight * convex_term
            diag["convex_term"] = convex_term.detach()

        # reduce
        red = reduction_override or self.reduction
        out = self._reduce(losses, red)

        if return_dict:
            diag["per_sample_loss"] = losses.detach()
            return out, diag
        return out

    @staticmethod
    def _reduce(loss: Tensor, reduction: str) -> Tensor:
        if reduction == "none":
            return loss
        if reduction == "mean":
            return loss.mean()
        if reduction == "sum":
            return loss.sum()
        raise ValueError(f"Unsupported reduction: {reduction!r}")

    def _mgiou_single(self, c1: Tensor, c2: Tensor) -> Tensor:
        axes = torch.cat((self._candidate_axes(c1), self._candidate_axes(c2)))
        giou1d = []
        for axis in axes:
            min1, max1 = (c1 @ axis).min(), (c1 @ axis).max()
            min2, max2 = (c2 @ axis).min(), (c2 @ axis).max()
            if self.fast_mode:
                numerator = torch.minimum(max1, max2) - torch.maximum(min1, min2)
                denominator = torch.maximum(max1, max2) - torch.minimum(min1, min2) + _PLUS_EPS
                giou1d.append(numerator / denominator)
            else:
                inter = (torch.minimum(max1, max2) - torch.maximum(min1, min2)).clamp(min=0.0)
                union = (max1 - min1) + (max2 - min2) - inter
                hull = (torch.maximum(max1, max2) - torch.minimum(min1, min2))
                giou1d.append(inter / union - (hull - union) / hull)
        return torch.mean(torch.stack(giou1d))

    @staticmethod
    def _candidate_axes(corners: Tensor) -> Tensor:
        center = corners.mean(dim=0, keepdim=True)
        angles = torch.atan2(corners[:, 1] - center[0, 1], corners[:, 0] - center[0, 0])
        corners = corners[angles.argsort()]  # clockwise
        edges = torch.vstack((corners[1:] - corners[:-1], corners[:1] - corners[-1:]))
        normals = torch.stack((edges[:, 1], -edges[:, 0]), dim=1)
        return normals

    @staticmethod
    def _convexity_loss(polygons: Tensor) -> Tensor:
        B, N, _ = polygons.shape
        v_prev = polygons[:, torch.arange(N) - 1]
        v_curr = polygons
        v_next = polygons[:, (torch.arange(N) + 1) % N]

        edge1 = v_prev - v_curr
        edge2 = v_next - v_curr
        cross = edge1[..., 0] * edge2[..., 1] - edge1[..., 1] * edge2[..., 0]

        sign_ref = torch.where(cross[:, 0:1].abs() <= _PLUS_EPS,
                               torch.ones_like(cross[:, 0:1]),
                               cross[:, 0:1]).sign()
        penalty = torch.clamp(-sign_ref * cross, min=0.0)
        return penalty.mean(dim=1)


In [ ]:
import torch
from torch import nn, Tensor
from typing import Optional

_EPS = 1e-6
_MIN_EDGE = 1e-3  # minimum allowed edge length

class MGIoU2DPlusHybrid(nn.Module):
    """
    Hybrid MGIoU loss with adaptive convex normalization.

    Combines:
      1. True polygonal MGIoU (multi-axis projection overlap)
      2. Optional convexity penalty normalized by polygon scale
      3. Safe numerical clipping and tanh saturation
      4. Automatic, robust adaptation of convex normalization exponent (no external input)

    --------------------------------------------------------------------------
    Adaptive normalization rule
    --------------------------------------------------------------------------
    The convex regularization term is normalized by mean edge length **L̄** raised to
    an exponent **p**:

        convex_term ∝ convex_raw / (L̄ ** p)

    Ideally, p ≈ 2 for scale invariance (area ∝ edge²), but during early training
    lower p (<2) gives stronger gradients and better shape correction.
    This module adapts **p** based on both model progress (MGIoU EMA) and
    geometric consistency (polygon scale spread).

    --------------------------------------------------------------------------
    Adaptive components
    --------------------------------------------------------------------------
    1. **MGIoU-driven schedule (training progress):**
       p = p_min + (p_max - p_min) * sigmoid(k * (mgiou_ema - thresh))

       - mgiou_ema : Exponential moving average of current MGIoU
       - p_min     : Lower bound for early stage (e.g. 1.2)
       - p_max     : Upper bound for late stage (theoretical scale-invariant 2.0)
       - k         : Steepness of sigmoid transition (default 5.0)
       - thresh    : MGIoU value where midpoint occurs (default 0.2)
       - ema_momentum : EMA smoothing (default 0.9)
       - dp_max    : Max per-step change to avoid instability (default 0.05)

    2. **Scale-robust normalization (geometry-driven):**
       Uses percentile-based edge length ratio instead of max/min to resist outliers.

       scale_ratio = (p95 / p05)
       adaptive_pow_scale = p_max - (p_max - p_min) * clamp(log10(scale_ratio) / decades_to_full, 0, 1)

       - decades_to_full : number of log10 decades to fully decay from p_max→p_min (default 3.0)
         meaning a 1000× scale difference saturates to p_min.
       - cap_ratio       : upper bound for ratio (default 1e4)

    The final exponent used:
        convex_norm_pow = mean(p_from_mgiou, p_from_scale)

    --------------------------------------------------------------------------
    Args:
        convex_weight: float
            Weight for convexity penalty. 0 disables it.
        reduction: str
            'none' | 'mean' | 'sum'
        loss_clip_max: float or None
            Optional clip for numeric stability.
        safe_saturate: bool
            Apply tanh to avoid extreme MGIoU ratios.
        convex_normalization_pow: float
            Initial base exponent for normalization.
        adaptive_convex_pow: bool
            Enable automatic adaptive scheduling.

        # MGIoU-based schedule parameters
        p_min, p_max, k, thresh, ema_momentum, dp_max

        # Scale-based adaptation parameters
        decades_to_full: float
            Controls how many log10 decades correspond to full transition.
        cap_ratio: float
            Hard cap for scale ratio.
        q_low, q_high: float
            Percentiles for robust scale ratio (default 0.05, 0.95)
    """

    def __init__(
        self,
        convex_weight: float = 0.25,
        reduction: str = "mean",
        loss_clip_max: Optional[float] = 2.0,
        safe_saturate: bool = True,
        convex_normalization_pow: float = 1.5,
        adaptive_convex_pow: bool = True,
        # MGIoU-based adaptive parameters
        p_min: float = 1.2,
        p_max: float = 2.0,
        k: float = 5.0,
        thresh: float = 0.2,
        ema_momentum: float = 0.9,
        dp_max: float = 0.05,
        # scale-based adaptive parameters
        decades_to_full: float = 3.0,
        cap_ratio: float = 1e4,
        q_low: float = 0.05,
        q_high: float = 0.95,
    ):
        super().__init__()
        if reduction not in {"none", "mean", "sum"}:
            raise ValueError("reduction must be 'none','mean' or 'sum'")

        self.convex_weight = float(convex_weight)
        self.reduction = reduction
        self.loss_clip_max = None if loss_clip_max is None else float(loss_clip_max)
        self.safe_saturate = bool(safe_saturate)
        self.convex_norm_pow = float(convex_normalization_pow)
        self.adaptive_convex_pow = adaptive_convex_pow

        # adaptive parameters
        self.p_min = p_min
        self.p_max = p_max
        self.k = k
        self.thresh = thresh
        self.ema_momentum = ema_momentum
        self.dp_max = dp_max
        self.decades_to_full = decades_to_full
        self.cap_ratio = cap_ratio
        self.q_low = q_low
        self.q_high = q_high

        # persistent states
        self.register_buffer("_mgiou_ema", torch.tensor(0.0))
        self.register_buffer("_prev_pow", torch.tensor(convex_normalization_pow))

    # ---------------------------------------------------- #
    #                   MAIN FORWARD                       #
    # ---------------------------------------------------- #
    def forward(
        self,
        pred: Tensor,      # [B,N,2]
        target: Tensor,    # [B,N,2]
        visible_mask: Optional[Tensor] = None,
        return_dict: bool = False,
    ):
        if visible_mask is not None:
            mask = visible_mask.bool().squeeze()
            pred, target = pred[mask], target[mask]

        # -------------------------------
        # Robust normalization of predictions
        # -------------------------------
        with torch.no_grad():
            pred_center = pred.mean(dim=1, keepdim=True)
            pred_scale = self._mean_edge_length(pred).clamp_min(_MIN_EDGE).unsqueeze(-1).unsqueeze(-1)
        pred_norm = (pred - pred_center) / pred_scale
        target_center = target.mean(dim=1, keepdim=True)
        target_scale = self._mean_edge_length(target).clamp_min(_EPS).unsqueeze(-1).unsqueeze(-1)
        target_norm = (target - target_center) / target_scale

        mgiou, diag = self._mgiou_batch(pred_norm, target_norm)
        loss = (1.0 - mgiou) * 0.5

        # -------------------------------
        # Adaptive convex normalization
        # -------------------------------
        if self.adaptive_convex_pow:
            with torch.no_grad():
                m = mgiou.mean().detach()
                self._mgiou_ema.mul_(self.ema_momentum).add_((1 - self.ema_momentum) * m)
                alpha = torch.sigmoid(self.k * (self._mgiou_ema - self.thresh))
                p_from_mgiou = self.p_min + (self.p_max - self.p_min) * alpha

                edge_scale = self._mean_edge_length(target)
                if edge_scale.numel() >= 6:
                    low = torch.quantile(edge_scale, self.q_low)
                    high = torch.quantile(edge_scale, self.q_high)
                else:
                    low, high = edge_scale.min(), edge_scale.max()
                ratio = (high / (low + _EPS)).clamp(1.0, self.cap_ratio)
                decades = torch.log10(ratio)
                norm = (decades / self.decades_to_full).clamp(0.0, 1.0)
                p_from_scale = self.p_max - (self.p_max - self.p_min) * norm

                p = 0.5 * (p_from_mgiou + p_from_scale)
                p = p.clamp(self._prev_pow - self.dp_max, self._prev_pow + self.dp_max)
                self._prev_pow.copy_(p)
                self.convex_norm_pow = float(p)

            diag["adaptive_pow"] = float(self.convex_norm_pow)
            diag["mgiou_ema"] = float(self._mgiou_ema)

        # -------------------------------
        # Convexity term (safe)
        # -------------------------------
        if self.convex_weight != 0.0:
            convex_mask = self._is_convex(target)
            convex_raw = self._convexity_loss(pred_norm)  # normalized pred
            denom = (edge_scale ** self.convex_norm_pow).clamp_min(_MIN_EDGE)
            convex_term = (convex_raw / denom) * self.convex_weight * convex_mask
            loss = loss + convex_term
            diag["convex_term"] = convex_term.detach()
            diag["convex_pow_used"] = float(self.convex_norm_pow)

        if self.loss_clip_max is not None:
            loss = torch.clamp(loss, 0.0, self.loss_clip_max)

        out = self._reduce(loss, self.reduction)
        if return_dict:
            diag.update({"per_sample_loss": loss.detach(), "mgiou": mgiou.detach()})
            return out, diag
        return out


    # ---------------------------------------------------- #
    #                MGIoU CORE ROUTINE                    #
    # ---------------------------------------------------- #
    def _mgiou_batch(self, c1: Tensor, c2: Tensor):
        scale = self._mean_edge_length(c2).unsqueeze(-1).unsqueeze(-1).clamp_min(_EPS)
        c1n = (c1 - c1.mean(dim=1, keepdim=True)) / scale
        c2n = (c2 - c2.mean(dim=1, keepdim=True)) / scale

        axes = torch.cat(
            (self._candidate_axes_batch(c1n), self._candidate_axes_batch(c2n)), dim=1
        )

        proj1 = torch.bmm(c1n, axes.transpose(1, 2))
        proj2 = torch.bmm(c2n, axes.transpose(1, 2))

        min1, _ = proj1.min(dim=1)
        max1, _ = proj1.max(dim=1)
        min2, _ = proj2.min(dim=1)
        max2, _ = proj2.max(dim=1)

        numerator = torch.minimum(max1, max2) - torch.maximum(min1, min2)
        denominator = torch.maximum(max1, max2) - torch.minimum(min1, min2)
        denominator = denominator.clamp_min(_EPS)
        numerator = torch.clamp(numerator, -denominator, denominator)
        giou1d = numerator / denominator

        if self.safe_saturate:
            giou1d = torch.tanh(giou1d)

        mgiou = giou1d.mean(dim=1)

        diag = {
            "giou1d_mean": giou1d.mean().detach(),
            "giou1d_std": giou1d.std().detach() if giou1d.numel() > 1 else torch.tensor(0.0),
            "edge_scale_mean": self._mean_edge_length(c2).detach(),
        }
        return mgiou.clamp(-1.0, 1.0), diag

    # ---------------------------------------------------- #
    #                  CONVEX HELPERS                      #
    # ---------------------------------------------------- #
    @staticmethod
    def _candidate_axes_batch(corners: Tensor) -> Tensor:
        center = corners.mean(dim=1, keepdim=True)
        rel = corners - center
        angles = torch.atan2(rel[..., 1], rel[..., 0])
        idx = angles.argsort(dim=1)
        sorted_corners = torch.gather(corners, 1, idx.unsqueeze(-1).expand_as(corners))
        edges = sorted_corners.roll(-1, dims=1) - sorted_corners
        normals = torch.stack((edges[..., 1], -edges[..., 0]), dim=-1)
        return normals

    @staticmethod
    def _convexity_loss(polygons: Tensor) -> Tensor:
        v_prev = polygons.roll(1, 1)
        v_next = polygons.roll(-1, 1)
        e1 = v_prev - polygons
        e2 = v_next - polygons
        cross = e1[..., 0] * e2[..., 1] - e1[..., 1] * e2[..., 0]
        sign_ref = torch.where(
            cross[:, 0:1].abs() <= _EPS, torch.ones_like(cross[:, 0:1]), cross[:, 0:1]
        ).sign()
        penalty = torch.clamp(-sign_ref * cross, 0.0)
        return penalty.mean(dim=1)

    @staticmethod
    def _is_convex(polygons: Tensor, eps: float = 1e-6) -> Tensor:
        v_prev = polygons.roll(1, 1)
        v_next = polygons.roll(-1, 1)
        e1 = v_prev - polygons
        e2 = v_next - polygons
        cross = e1[..., 0] * e2[..., 1] - e1[..., 1] * e2[..., 0]
        sign_ref = torch.where(
            cross[:, 0:1].abs() <= eps, torch.ones_like(cross[:, 0:1]), cross[:, 0:1]
        ).sign()
        same_sign = (cross * sign_ref > -eps).all(dim=1)
        return same_sign.float()

    @staticmethod
    def _mean_edge_length(polygons: Tensor) -> Tensor:
        edges = polygons.roll(-1, dims=1) - polygons
        return edges.norm(dim=-1).mean(dim=1).clamp(min=_EPS)

    @staticmethod
    def _reduce(loss: Tensor, reduction: str) -> Tensor:
        if reduction == "none":
            return loss
        if reduction == "mean":
            return loss.mean()
        if reduction == "sum":
            return loss.sum()
        raise ValueError(f"Unsupported reduction: {reduction!r}")



In [105]:
import torch
import time
from torch import nn
from typing import Callable

# ----------------------
# CONFIGURABLE BENCHMARK
# ----------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float32

batch_sizes = [1, 4, 16, 32]          # different batch sizes
num_vertices_list = [4, 6, 8, 16]     # polygon complexity
num_trials = 3                         # repeat for avg runtime

# Hard/edge cases generator
def generate_polygons(batch: int, vertices: int, device=DEVICE, dtype=DTYPE):
    # Random polygons in [0,1]
    polys = torch.rand(batch, vertices, 2, device=device, dtype=dtype)
    
    # Introduce degenerate / extreme cases randomly
    for i in range(batch):
        if torch.rand(1) < 0.2:
            # nearly collinear
            polys[i] = torch.linspace(0,1,vertices, device=device).unsqueeze(-1).repeat(1,2)
        if torch.rand(1) < 0.1:
            # duplicate points
            polys[i, 1] = polys[i, 0]
        if torch.rand(1) < 0.05:
            # zeros
            polys[i] = 0.0
    return polys

# Benchmark runner
def benchmark_loss_grad(model_class: Callable, convex_weight=0.25):
    results = []

    for batch in batch_sizes:
        for verts in num_vertices_list:
            # generate inputs
            pred = generate_polygons(batch, verts)
            target = generate_polygons(batch, verts)

            pred.requires_grad_()
            
            # instantiate model
            model = model_class(convex_weight=convex_weight).to(DEVICE)

            # timing + forward
            runtimes = []
            for _ in range(num_trials):
                start = time.time()
                loss, diag = model(pred, target, return_dict=True)
                torch.cuda.synchronize() if DEVICE=="cuda" else None
                end = time.time()
                runtimes.append(end - start)
            
            # backward + grad norm
            loss_sum = loss.sum()
            loss_sum.backward()
            grad_norm = pred.grad.norm().item()

            results.append({
                "model": model_class.__name__,
                "batch": batch,
                "vertices": verts,
                "loss_mean": loss.mean().item(),
                "loss_std": loss.std().item(),
                "grad_norm": grad_norm,
                "runtime_avg_s": sum(runtimes)/len(runtimes),
                "diag_keys": list(diag.keys())
            })
    return results

In [113]:

import pandas as pd

# compare both models
results_plus = benchmark_loss_grad(MGIoU2DPlus, convex_weight=0.25)
results_hybrid = benchmark_loss_grad(MGIoU2DPlusHybrid, convex_weight=0.25)

df = pd.DataFrame(results_plus + results_hybrid)


/tmp/ipykernel_5299/3998304421.py:68: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1857.)
  "loss_std": loss.std().item(),


In [114]:
df

,model,batch,vertices,loss_mean,loss_std,grad_norm,runtime_avg_s,diag_keys
0,MGIoU2DPlus,1,4,NaN,NaN,NaN,0.004504,"[mgiou, convex_term, per_sample_loss]"
1,MGIoU2DPlus,1,6,0.265442,NaN,0.453401,0.004991,"[mgiou, convex_term, per_sample_loss]"
2,MGIoU2DPlus,1,8,NaN,NaN,NaN,0.005726,"[mgiou, convex_term, per_sample_loss]"
3,MGIoU2DPlus,1,16,0.099923,NaN,0.479053,0.011569,"[mgiou, convex_term, per_sample_loss]"
4,MGIoU2DPlus,4,4,NaN,NaN,NaN,0.003997,"[mgiou, convex_term, per_sample_loss]"
5,MGIoU2DPlus,4,6,NaN,NaN,NaN,0.005524,"[mgiou, convex_term, per_sample_loss]"
6,MGIoU2DPlus,4,8,NaN,NaN,NaN,0.007166,"[mgiou, convex_term, per_sample_loss]"
7,MGIoU2DPlus,4,16,NaN,NaN,NaN,0.014392,"[mgiou, convex_term, per_sample_loss]"
8,MGIoU2DPlus,16,4,NaN,NaN,NaN,0.003958,"[mgiou, convex_term, per_sample_loss]"
9,MGIoU2DPlus,16,6,NaN,NaN,NaN,0.004823,"[mgiou, convex_term, per_sample_loss]"


In [111]:
df

,model,batch,vertices,loss_mean,loss_std,grad_norm,runtime_avg_s,diag_keys
0,MGIoU2DPlus,1,4,0.562311,NaN,1.333730,0.004456,"[mgiou, convex_term, per_sample_loss]"
1,MGIoU2DPlus,1,6,0.211020,NaN,0.489262,0.005010,"[mgiou, convex_term, per_sample_loss]"
2,MGIoU2DPlus,1,8,0.192046,NaN,0.333614,0.006245,"[mgiou, convex_term, per_sample_loss]"
3,MGIoU2DPlus,1,16,0.178138,NaN,0.369451,0.011842,"[mgiou, convex_term, per_sample_loss]"
4,MGIoU2DPlus,4,4,NaN,NaN,NaN,0.004837,"[mgiou, convex_term, per_sample_loss]"
5,MGIoU2DPlus,4,6,0.309356,NaN,0.304428,0.005354,"[mgiou, convex_term, per_sample_loss]"
6,MGIoU2DPlus,4,8,NaN,NaN,NaN,0.006502,"[mgiou, convex_term, per_sample_loss]"
7,MGIoU2DPlus,4,16,0.146470,NaN,0.307050,0.011504,"[mgiou, convex_term, per_sample_loss]"
8,MGIoU2DPlus,16,4,NaN,NaN,NaN,0.004382,"[mgiou, convex_term, per_sample_loss]"
9,MGIoU2DPlus,16,6,NaN,NaN,NaN,0.005288,"[mgiou, convex_term, per_sample_loss]"


In [108]:
df

,model,batch,vertices,loss_mean,loss_std,grad_norm,runtime_avg_s,diag_keys
0,MGIoU2DPlus,1,4,NaN,NaN,NaN,0.005110,"[mgiou, convex_term, per_sample_loss]"
1,MGIoU2DPlus,1,6,0.097976,NaN,0.366121,0.006600,"[mgiou, convex_term, per_sample_loss]"
2,MGIoU2DPlus,1,8,0.255606,NaN,1.169231,0.007420,"[mgiou, convex_term, per_sample_loss]"
3,MGIoU2DPlus,1,16,0.116625,NaN,0.359974,0.012540,"[mgiou, convex_term, per_sample_loss]"
4,MGIoU2DPlus,4,4,0.322299,NaN,0.230722,0.004332,"[mgiou, convex_term, per_sample_loss]"
5,MGIoU2DPlus,4,6,0.253121,NaN,0.205108,0.006195,"[mgiou, convex_term, per_sample_loss]"
6,MGIoU2DPlus,4,8,0.232595,NaN,0.142631,0.007950,"[mgiou, convex_term, per_sample_loss]"
7,MGIoU2DPlus,4,16,0.238224,NaN,0.337288,0.012922,"[mgiou, convex_term, per_sample_loss]"
8,MGIoU2DPlus,16,4,NaN,NaN,NaN,0.008276,"[mgiou, convex_term, per_sample_loss]"
9,MGIoU2DPlus,16,6,NaN,NaN,NaN,0.006136,"[mgiou, convex_term, per_sample_loss]"
